In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
import pandas as pd 

#### // 读取数据

In [ ]:
train = pd.read_csv("data/train.csv")

In [ ]:
images = train.iloc[:,1:].values

In [ ]:
labels_flat = train.iloc[:,:1].values.ravel()

In [ ]:
images = images.astype(np.float)

In [ ]:
images = np.multiply(images, 1.0/255.0)

In [ ]:
image_size = images.shape[1]

In [ ]:
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

In [ ]:
x = tf.placeholder("float", shape=[None, image_size])

In [ ]:
labels_count = np.unique(labels_flat).shape[0]

In [ ]:
y = tf.placeholder("float", shape=[None, labels_count])

In [ ]:
def dense_to_one_hot(label_dense,num_classes):
    num_lables = label_dense.shape[0]
    index_offset = np.arange(num_lables) *num_classes
    label_one_hot = np.zeros((num_lables, num_classes))
    label_one_hot.flat[index_offset + label_dense.ravel()] = 1
    return label_one_hot

In [ ]:
labels = dense_to_one_hot(labels_flat,labels_count)

In [ ]:
labels = labels.astype(np.uint8)

#### // 数据集分组

In [ ]:
VALIDATION_SIZE = 2000

In [ ]:
validation_images = images[:VALIDATION_SIZE]

In [ ]:
validation_labels = labels[:VALIDATION_SIZE]

In [ ]:
train_images = images[VALIDATION_SIZE:]

In [ ]:
train_labels = labels[VALIDATION_SIZE:]

####  // 模型

In [ ]:
def weight_varible(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [ ]:
def bias_varible(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1], padding='SAME')

In [ ]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

In [ ]:
x_image = tf.reshape(x,[-1,28,28,1])

In [ ]:
w_conv1 = weight_varible([3,3,1,32])

In [ ]:
b_conv1 = bias_varible([32])

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)

In [ ]:
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
w_conv2 = weight_varible([6,6,32,64])

In [ ]:
b_conv2 = bias_varible([64])

In [ ]:
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)

In [ ]:
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])

In [ ]:
w_fc1 = weight_varible([7*7*64,1024])

In [ ]:
b_fc1 = bias_varible([1024])

In [ ]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

In [ ]:
keep_prob = tf.placeholder(tf.float32)

In [ ]:
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [ ]:
w_fc2 = weight_varible([1024,10])

In [ ]:
b_fc2 = bias_varible([10])

In [ ]:
y_conv = tf.matmul(h_fc1_drop,w_fc2)+b_fc2

#### // 创建损失函数

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=y_conv))

In [ ]:
train_step = tf.train.AdadeltaOptimizer(learning_rate=0.1).minimize(loss)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_conv,1))

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
global_step = tf.Variable(0,name='global_step',trainable=False)

In [ ]:
saver= tf.train.Saver()

In [ ]:
init = tf.global_variables_initializer()

#### // train

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for enpoch in range(20):
        batch_size = 100
        n_batch = len(train_images)/batch_size
        for batch in range(n_batch):
            print("----------->round:"+str(enpoch+1) + "/20, rate:"+str(batch+1) + "/" + str(n_batch))
            batch_x = train_images[batch*batch_size:(batch+1)*batch_size]
            batch_y = train_labels[batch*batch_size:(batch+1)*batch_size]
            sess.run(train_step,feed_dict={x:validation_images,y:validation_labels,keep_prob:0.5})
            
        accuracy_n = sess.run(accuracy,feed_dict={x:validation_images,y:validation_labels,keep_prob:1.0})
        print("第"+str(enpoch+1)+"轮，准确度为："+str(accuracy_n))
        global_step.assign(enpoch).eval()
        saver.save(sess,"/Users/max/Desktop/TensorFlow/model.ckpt",global_step=global_step)